In [3]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough

memory = ConversationBufferWindowMemory(return_messages=True, memory_key="history", k=5)

chat = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
    streaming=True,
)

loader = UnstructuredFileLoader("input/text.txt")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",
    chunk_size=300,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)

cache_dir = LocalFileStore("./.cache")
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vector_store = FAISS.from_documents(docs, cached_embeddings)
retriever = vector_store.as_retriever()


def load_memory(_):
    return memory.load_memory_variables({})["history"]


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | chat
)


def invoke_chain(question: str):
    result = chain.invoke(question)
    print(result.content)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

Created a chunk of size 326, which is longer than the specified 300
Created a chunk of size 319, which is longer than the specified 300
Created a chunk of size 317, which is longer than the specified 300
Created a chunk of size 303, which is longer than the specified 300
Created a chunk of size 304, which is longer than the specified 300
Created a chunk of size 312, which is longer than the specified 300
Created a chunk of size 326, which is longer than the specified 300


In [4]:
invoke_chain("Is Aaronson guilty?")

According to the context, Aaronson is guilty of the crimes he was charged with, as stated: "Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with."


In [5]:
invoke_chain("What message did he write on the table?")

He wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" on the table.


In [6]:
invoke_chain("Who is Julia?")

Julia is a character who is loved by the narrator. The narrator experiences a strong emotional connection to her, feeling that she is not only with him but also inside him at one point. She is someone he cares for deeply, and he expresses a desire to protect her during moments of distress.
